# Spaceship Titanic

Our goal is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly.


## File and Data Field Descriptions

* **train.csv** - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
    * PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
    * HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
    * CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
    * Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
    * Destination - The planet the passenger will be debarking to.
    * Age - The age of the passenger.
    * VIP - Whether the passenger has paid for special VIP service during the voyage.
    * RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
    * Name - The first and last names of the passenger.
    * Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.
* **test.csv** - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of Transported for the passengers in this set.
* **sample_submission.csv** - A submission file in the correct format.
    * PassengerId - Id for each passenger in the test set.
    * Transported - The target. For each passenger, predict either True or False.

## Libraries imports

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1) Dataset loading and preprocess

First of all, we load the training and test sets.

In [2]:
# Load a dataset into a Pandas Dataframe
# Try to load the dataset from Kaggle, if not found, load from local directory
try:
    train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
    test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
except FileNotFoundError:
    train_df = pd.read_csv('kaggle/input/spaceship-titanic/train.csv')
    test_df = pd.read_csv('kaggle/input/spaceship-titanic/test.csv')

print("Full train dataset shape is {}".format(train_df.shape))
print("Full test dataset shape is {}".format(test_df.shape))

Full train dataset shape is (8693, 14)
Full test dataset shape is (4277, 13)


In [3]:
# I split the datasets into features (X) and tag (Y)
train_x = train_df.drop(columns=['Transported'])
train_y = train_df['Transported'].astype(int)  # Convert boolean to int (0 or 1)

test_x = test_df

To evaluate the different models used, I split the training set into train and validation, giving a 10% of the samples to the validation set. I set that percentage in order to have a dataset big enough to evaluate the models.

In [4]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=0)

print("Train dataset shape is {}".format(train_x.shape))
print("Validation dataset shape is {}".format(val_x.shape))
train_x.head()

Train dataset shape is (7823, 13)
Validation dataset shape is (870, 13)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
4132,4408_01,Mars,True,F/906/P,TRAPPIST-1e,75.0,False,0.0,0.0,0.0,0.0,0.0,Pich Knike
7217,7710_01,Europa,False,B/253/P,TRAPPIST-1e,27.0,False,118.0,1769.0,4127.0,118.0,619.0,Chabih Eguing
7216,7709_02,Earth,True,G/1238/P,PSO J318.5-22,24.0,False,0.0,NaN,0.0,0.0,0.0,Lerome Sweett
7968,8512_01,Earth,False,F/1637/S,55 Cancri e,48.0,False,0.0,717.0,0.0,0.0,10.0,Verly Flyncharlan
50,0052_01,Earth,False,G/6/S,TRAPPIST-1e,NaN,False,4.0,0.0,2.0,4683.0,0.0,Elaney Hubbarton


After separate a validation set from our training set, the next step is to preprocess the dataset, to do that I will check each individual feature.

### PassengerId

This variable represents a unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.

At the begining it seems to be a variant without information to solve the problem, something like a name, however as it said, we can extract the number of members that each group has and that can be something relevant.
However to know the number of members a group has, we have to use the whole dataset, trining + validation + test sets. To do this would be a problem in most cases because we are not sure that our dataset contain all the existent samples, however for this task, the description tells us that we have about two-thirds (~8700) of the passengers as the training set and THE REMAINING one-third (~4300) of the passengers as test, so we got at least partial information about all the passengers.

In [5]:
# Check for missing values in PassengerId
print("Missing values in PassengerId:")
print(f"train_x: {train_x['PassengerId'].isnull().sum()}")
print(f"test_x: {test_x['PassengerId'].isnull().sum()}")
print(f"val_x: {val_x['PassengerId'].isnull().sum()}")

# Extract group identifier (gggg) from PassengerId
train_x['Group'] = train_x['PassengerId'].str.split('_').str[0]
test_x['Group'] = test_x['PassengerId'].str.split('_').str[0]
val_x['Group'] = val_x['PassengerId'].str.split('_').str[0]

# Combine all dataframes to calculate group sizes across all datasets
combined_df = pd.concat([train_x, val_x, test_x])

# Calculate the total number of members in each group
group_sizes = combined_df['Group'].value_counts()

# Add the numMembers column to each dataframe
train_x['numMembers'] = train_x['Group'].map(group_sizes)
val_x['numMembers'] = val_x['Group'].map(group_sizes)
test_x['numMembers'] = test_x['Group'].map(group_sizes)

# Display the updated dataframes
print("Updated train_x:")
print(train_x[['PassengerId', 'Group', 'numMembers']].head())

print("\nUpdated val_x:")
print(val_x[['PassengerId', 'Group', 'numMembers']].head())

print("\nUpdated test_x:")
print(test_x[['PassengerId', 'Group', 'numMembers']].head())


Missing values in PassengerId:
train_x: 0
test_x: 0
val_x: 0
Updated train_x:
     PassengerId Group  numMembers
4132     4408_01  4408           1
7217     7710_01  7710           1
7216     7709_02  7709           2
7968     8512_01  8512           1
50       0052_01  0052           1

Updated val_x:
     PassengerId Group  numMembers
3601     3868_05  3868           7
6057     6405_02  6405           4
2797     3021_01  3021           2
7110     7578_01  7578           1
8579     9158_01  9158           1

Updated test_x:
  PassengerId Group  numMembers
0     0013_01  0013           1
1     0018_01  0018           1
2     0019_01  0019           1
3     0021_01  0021           1
4     0023_01  0023           1


Now we can remove the columns PassengerId and Group, that don't give us any more information, however I will wait and remove all the useless columns after analyze all the features.

### HomePlanet 
The planet the passenger departed from, typically their planet of permanent residence.

This is a categorical feature, so we will need to encode it in some way in order to give the information to the models.

In [6]:
# First I check how many different values we have in this variant (HomePlanet)
print("HomePlanet unique values: ", train_x['HomePlanet'].unique())

# And also if there are any missing values and how many
print("Missing values in HomePlanet: ", train_x['HomePlanet'].isnull().sum(), f"({train_x['HomePlanet'].isnull().sum()/len(train_x)*100:.2f}%)")

HomePlanet unique values:  ['Mars' 'Europa' 'Earth' nan]
Missing values in HomePlanet:  174 (2.22%)


First of all, let's solve the issue with the missing values, we can assume that members of the same group depart from the same planet, so the first approach will be to assign the existing value of one member of the group to other members that have a null value in the HomePlanet variable.

In [34]:
# Check if all members of the same group have the same HomePlanet
group_destination_check = train_x.groupby('Group')['HomePlanet'].nunique()

# Find groups with more than one unique HomePlanet
inconsistent_groups = group_destination_check[group_destination_check > 1]

# Print the result
if inconsistent_groups.empty:
    print("All members of the same group have the same HomePlanet.")
else:
    print("Some groups have inconsistent HomePlanet:")
    print(inconsistent_groups)

All members of the same group have the same HomePlanet.


In [7]:
combined_df = pd.concat([train_x, val_x, test_x])

# Fill missing HomePlanet values based on the group
combined_df['HomePlanet'] = combined_df.groupby('Group')['HomePlanet'].transform(lambda x: x.ffill().bfill())

# Update train_x, val_x, and test_x with the filled values from combined_df based on PassengerId
train_x['HomePlanet'] = train_x['PassengerId'].map(combined_df.set_index('PassengerId')['HomePlanet'])
val_x['HomePlanet'] = val_x['PassengerId'].map(combined_df.set_index('PassengerId')['HomePlanet'])
test_x['HomePlanet'] = test_x['PassengerId'].map(combined_df.set_index('PassengerId')['HomePlanet'])

# Verify if there are still missing values in HomePlanet
print("Missing values in HomePlanet after filling:")
print(f"train_x: {train_x['HomePlanet'].isnull().sum()}")
print(f"val_x: {val_x['HomePlanet'].isnull().sum()}")
print(f"test_x: {test_x['HomePlanet'].isnull().sum()}")

C:\Users\javie\AppData\Local\Temp\ipykernel_284\2562908295.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['HomePlanet'] = combined_df.groupby('Group')['HomePlanet'].transform(lambda x: x.ffill().bfill())


Missing values in HomePlanet after filling:
train_x: 96
val_x: 15
test_x: 46


In [8]:
# Check an example to see that everything is correct, in this case the group 0064 come from Mars
combined_df[combined_df['Name'] == 'Colatz Keen']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Group,numMembers
59,0064_02,Mars,True,E/3/S,TRAPPIST-1e,33.0,False,0.0,0.0,NaN,0.0,0.0,Colatz Keen,0064,2


We filled ~50% of the empty values based on the training set, however we still have a little bit more than a 1% of empty entries, so now I will check the class distribution, maybe there is one popular class and I can simply give that class value to the empty entries.

In [9]:
# Class distribution for the HomePlanet column
homeplanet_distribution = train_x['HomePlanet'].value_counts(dropna=False)
homeplanet_percentages = (train_x['HomePlanet'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("HomePlanet distribution (counts):")
print(homeplanet_distribution)
print("\nHomePlanet distribution (percentages):")
print(homeplanet_percentages)

HomePlanet distribution (counts):
HomePlanet
Earth     4151
Europa    1962
Mars      1614
NaN         96
Name: count, dtype: int64

HomePlanet distribution (percentages):
HomePlanet
Earth     53.06
Europa    25.08
Mars      20.63
NaN        1.23
Name: proportion, dtype: float64


Seeing the distribution, if I assign Earth to the empty values, I will aim ~53% of the cases, taking into account that we have 1.23% of empty values, I will have ~0.6% of incorrect values with this approach, this is a very low percentage, so I go with this plan.

In [10]:
# Fill NaN values in the HomePlanet column with 'Earth'
train_x['HomePlanet'] = train_x['HomePlanet'].fillna('Earth')
val_x['HomePlanet'] = val_x['HomePlanet'].fillna('Earth')
test_x['HomePlanet'] = test_x['HomePlanet'].fillna('Earth')

# Verify if there are still missing values in HomePlanet
print("Missing values in HomePlanet after filling with 'Earth':")
print(f"train_x: {train_x['HomePlanet'].isnull().sum()}")
print(f"val_x: {val_x['HomePlanet'].isnull().sum()}")
print(f"test_x: {test_x['HomePlanet'].isnull().sum()}")

Missing values in HomePlanet after filling with 'Earth':
train_x: 0
val_x: 0
test_x: 0


Finally, in order to train the model with this feature, we need to encode it, to do that I will use one-hot encoding that will add 1 more variable to the problem.

The idea is to pass from HomePlanet to two boolean variables, isHomeEarth and isHomeEuropa, if both variants are False we still have the info that the HomePlanet is Mars without having to explicitly saving into another variant. With this approach I am assuming that the only possible HomePlanets are Earth, Mars and Europa, which is not crazy to say seeing the class distribution in the training set. 

In [11]:
# One-hot encoding for HomePlanet
train_x['isHomeEarth'] = (train_x['HomePlanet'] == 'Earth').astype(int)
train_x['isHomeEuropa'] = (train_x['HomePlanet'] == 'Europa').astype(int)

val_x['isHomeEarth'] = (val_x['HomePlanet'] == 'Earth').astype(int)
val_x['isHomeEuropa'] = (val_x['HomePlanet'] == 'Europa').astype(int)

test_x['isHomeEarth'] = (test_x['HomePlanet'] == 'Earth').astype(int)
test_x['isHomeEuropa'] = (test_x['HomePlanet'] == 'Europa').astype(int)

# Verify the new columns
train_x[['HomePlanet', 'isHomeEarth', 'isHomeEuropa']].head()

,HomePlanet,isHomeEarth,isHomeEuropa
4132,Mars,0,0
7217,Europa,0,1
7216,Earth,1,0
7968,Earth,1,0
50,Earth,1,0


### CryoSleep 
Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

This is a boolean variable, so we only have to cast it into integer, however first I need to check if there are any empty value and the class distribution.

In [ ]:
# Print the distribution of CryoSleep
cryosleep_distribution = train_x['CryoSleep'].value_counts(dropna=False)
cryosleep_percentages = (train_x['CryoSleep'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("CryoSleep distribution (counts):")
print(cryosleep_distribution)
print("\nCryoSleep distribution (percentages):")
print(cryosleep_percentages)

print("Missing values in CryoSleep: ", train_x['CryoSleep'].isnull().sum(), f"({train_x['CryoSleep'].isnull().sum()/len(train_x)*100:.2f}%)")

CryoSleep distribution (counts):
CryoSleep
False    4898
True     2731
NaN       194
Name: count, dtype: int64
\CryoSleep distribution (percentages):
CryoSleep
False    62.61
True     34.91
NaN       2.48
Name: proportion, dtype: float64
Missing values in CryoSleep:  194 (2.48%)


Similar than with the HomePlanet, we have ~2.5% of empty values.

In this case, we have the statement where if a passenger is in cryosleep, he is confined inside his cabin, so he won't spend any money on services, that means that variables RoomService, FoodCourt, ShoppingMall, Spa and VRDeck will be all 0.

To confirm this, lets check:
* When CryoSleep == True, sum(RoomService, FoodCourt, ShoppingMall, Spa and VRDeck) == 0.0
* When CryoSleep == False, sum(RoomService, FoodCourt, ShoppingMall, Spa and VRDeck) > 0.0

To do this check I will exclude samples with any of this fields empty.

In [13]:
# Exclude samples with any of the relevant fields empty
filtered_df = train_x.dropna(subset=['CryoSleep', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])

# Check the conditions
cryosleep_true = filtered_df[filtered_df['CryoSleep'] == True]
cryosleep_false = filtered_df[filtered_df['CryoSleep'] == False]

cryosleep_true_condition = cryosleep_true[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1).eq(0.0)
cryosleep_false_condition = cryosleep_false[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1).gt(0.0)

# Print the results
print("CryoSleep == True and all services == 0.0:")
print(cryosleep_true_condition.all())
print(f"Number of samples: {len(cryosleep_true)}")
if not cryosleep_true_condition.all():
    print("\nRows where CryoSleep == True and all services != 0.0:")
    print(cryosleep_true[~cryosleep_true_condition])

print("\nCryoSleep == False and sum of services > 0.0:")
print(cryosleep_false_condition.all())
print(f"Number of samples: {len(cryosleep_false)}")
if not cryosleep_false_condition.all():
    print("\nRows where CryoSleep == False and sum of services == 0.0:")
    print(cryosleep_false[~cryosleep_false_condition])

CryoSleep == True and all services == 0.0:
True
Number of samples: 2413

CryoSleep == False and sum of services > 0.0:
False
Number of samples: 4408

Rows where CryoSleep == False and sum of services == 0.0:
     PassengerId HomePlanet CryoSleep     Cabin  Destination   Age    VIP  \
5554     5922_01      Earth     False   G/960/P  TRAPPIST-1e  32.0  False   
8440     9013_03      Earth     False  G/1461/P  55 Cancri e   0.0  False   
396      0436_01      Earth     False    G/63/S  55 Cancri e  10.0  False   
5792     6137_01      Earth     False   G/994/S  TRAPPIST-1e   2.0  False   
2948     3195_05      Earth     False       NaN  55 Cancri e   0.0  False   
...          ...        ...       ...       ...          ...   ...    ...   
4305     4592_01       Mars     False   F/862/S  TRAPPIST-1e   9.0  False   
7316     7830_04      Earth     False  G/1268/S  TRAPPIST-1e   0.0  False   
3455     3715_03      Earth     False   G/602/P  TRAPPIST-1e   2.0  False   
2418     2595_01      

Well, with the obtained results we can assume that if a passenger has spent something, he is not on cryosleep, however we CANNOT assume that if a passenger didn't spend anything, he is on cryosleep.

So let's do a recap, taking into account the distribution (~62.5% False -- ~35% True), the most probable is that inside that ~2.5% of empty values this distribution also apply, so let's apply the rule where if a passenger has spent something, he is not on cryosleep and see the new distribution to evaluate the next steps.

In [14]:
# When 'CryoSleep' is nan, if a passenger has spent something on 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa' or 'VRDeck', he is not on cryosleep, so we can fill 'CryoSleep' as False
train_x.loc[train_x['CryoSleep'].isnull() & (train_x[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) > 0.0), 'CryoSleep'] = False
val_x.loc[val_x['CryoSleep'].isnull() & (val_x[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) > 0.0), 'CryoSleep'] = False
test_x.loc[train_x['CryoSleep'].isnull() & (test_x[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) > 0.0), 'CryoSleep'] = False

# Print the updated distribution of CryoSleep
cryosleep_distribution = train_x['CryoSleep'].value_counts(dropna=False)
cryosleep_percentages = (train_x['CryoSleep'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("CryoSleep distribution (counts):")
print(cryosleep_distribution)
print("\CryoSleep distribution (percentages):")
print(cryosleep_percentages)

print("Missing values in CryoSleep: ", train_x['CryoSleep'].isnull().sum(), f"({train_x['CryoSleep'].isnull().sum()/len(train_x)*100:.2f}%)")

CryoSleep distribution (counts):
CryoSleep
False    5006
True     2731
NaN        86
Name: count, dtype: int64
\CryoSleep distribution (percentages):
CryoSleep
False    63.99
True     34.91
NaN       1.10
Name: proportion, dtype: float64
Missing values in CryoSleep:  86 (1.10%)


We had 194 NaN values and now we have only 86, meaning that from that set of 194 samples, the 55.67% of the samples are not CryoSleep and the remain percentage is still unknown, taking into account that approx. the 62.61% of the samples inside a set are not CryoSleep, there is still a 6.94% of the samples that can not be CryoSleep, this means that there are ~13 samples that are not CryoSleep, but didn't spend a penny. In total percentages this means that from the 7823 samples that the training set has, a ~0.17% of them will have this field wrong. So we can take the risk of assign CryoSleep == True to the rest of NaN values.

In [15]:
# Fill NaN values in the CryoSleep column with True
train_x['CryoSleep'] = train_x['CryoSleep'].fillna(True)
val_x['CryoSleep'] = val_x['CryoSleep'].fillna(True)
test_x['CryoSleep'] = test_x['CryoSleep'].fillna(True)

# Verify if there are still missing values in CryoSleep
print("Missing values in CryoSleep after filling with True:")
print(f"train_x: {train_x['CryoSleep'].isnull().sum()}")
print(f"val_x: {val_x['CryoSleep'].isnull().sum()}")
print(f"test_x: {test_x['CryoSleep'].isnull().sum()}")

Missing values in CryoSleep after filling with True:
train_x: 0
val_x: 0
test_x: 0


C:\Users\javie\AppData\Local\Temp\ipykernel_284\1584031860.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_x['CryoSleep'] = train_x['CryoSleep'].fillna(True)
C:\Users\javie\AppData\Local\Temp\ipykernel_284\1584031860.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_x['CryoSleep'] = val_x['CryoSleep'].fillna(True)
C:\Users\javie\AppData\Local\Temp\ipykernel_284\1584031860.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) 

Finally, let's cast the boolean values into integers.

In [16]:
train_x['CryoSleep'] = train_x['CryoSleep'].astype(int)
val_x['CryoSleep'] = val_x['CryoSleep'].astype(int)
test_x['CryoSleep'] = test_x['CryoSleep'].astype(int)

train_x.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Group,numMembers,isHomeEarth,isHomeEuropa
4132,4408_01,Mars,1,F/906/P,TRAPPIST-1e,75.0,False,0.0,0.0,0.0,0.0,0.0,Pich Knike,4408,1,0,0
7217,7710_01,Europa,0,B/253/P,TRAPPIST-1e,27.0,False,118.0,1769.0,4127.0,118.0,619.0,Chabih Eguing,7710,1,0,1
7216,7709_02,Earth,1,G/1238/P,PSO J318.5-22,24.0,False,0.0,NaN,0.0,0.0,0.0,Lerome Sweett,7709,2,1,0
7968,8512_01,Earth,0,F/1637/S,55 Cancri e,48.0,False,0.0,717.0,0.0,0.0,10.0,Verly Flyncharlan,8512,1,1,0
50,0052_01,Earth,0,G/6/S,TRAPPIST-1e,NaN,False,4.0,0.0,2.0,4683.0,0.0,Elaney Hubbarton,0052,1,1,0


### Cabin
The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard. So first let's separate each value and save them in different columns.

In [20]:
# Split the Cabin column into deck, num, and side
train_x[['CabinDeck', 'CabinNum', 'CabinSide']] = train_x['Cabin'].str.split('/', expand=True)
val_x[['CabinDeck', 'CabinNum', 'CabinSide']] = val_x['Cabin'].str.split('/', expand=True)
test_x[['CabinDeck', 'CabinNum', 'CabinSide']] = test_x['Cabin'].str.split('/', expand=True)

# Convert CabinNum to numeric for potential numerical analysis
train_x['CabinNum'] = pd.to_numeric(train_x['CabinNum'], errors='coerce').astype('Int64')
val_x['CabinNum'] = pd.to_numeric(val_x['CabinNum'], errors='coerce').astype('Int64')
test_x['CabinNum'] = pd.to_numeric(test_x['CabinNum'], errors='coerce').astype('Int64')

# Display the updated train_x dataframe
train_x[['Cabin', 'CabinDeck', 'CabinNum', 'CabinSide']].head()

,Cabin,CabinDeck,CabinNum,CabinSide
4132,F/906/P,F,906,P
7217,B/253/P,B,253,P
7216,G/1238/P,G,1238,P
7968,F/1637/S,F,1637,S
50,G/6/S,G,6,S


With the data divided in deck/num/side, let's now see the distribution and the nan values.

In [21]:
# Check the distribution and missing values for CabinDeck, CabinNum, and CabinSide
for column in ['CabinDeck', 'CabinNum', 'CabinSide']:
    print(f"Distribution for {column}:")
    print(train_x[column].value_counts(dropna=False))
    print(f"\nMissing values in {column}: {train_x[column].isnull().sum()} ({train_x[column].isnull().mean() * 100:.2f}%)\n")

Distribution for CabinDeck:
CabinDeck
F      2504
G      2296
E       799
B       707
C       672
D       433
A       232
NaN     175
T         5
Name: count, dtype: int64

Missing values in CabinDeck: 175 (2.24%)

Distribution for CabinNum:
CabinNum
<NA>    175
82       28
19       21
56       20
86       20
       ... 
1607      1
1133      1
1669      1
1866      1
1519      1
Name: count, Length: 1781, dtype: Int64

Missing values in CabinNum: 175 (2.24%)

Distribution for CabinSide:
CabinSide
S      3881
P      3767
NaN     175
Name: count, dtype: int64

Missing values in CabinSide: 175 (2.24%)



In [ ]:
# Distribution of CabinDeck based on CabinSide
for side in ['S', 'P']:
    print(f"Distribution for CabinSide = {side}:\n")
    
    # CabinDeck distribution
    cabin_deck_distribution = train_x[train_x['CabinSide'] == side]['CabinDeck'].value_counts(dropna=False)
    print(f"CabinDeck distribution:\n{cabin_deck_distribution}\n")

Distribution for CabinSide = S:

CabinDeck distribution:
CabinDeck
F    1211
G    1164
E     404
B     393
C     370
D     212
A     126
T       1
Name: count, dtype: int64

Distribution for CabinSide = P:

CabinDeck distribution:
CabinDeck
F    1293
G    1132
E     395
B     314
C     302
D     221
A     106
T       4
Name: count, dtype: int64



In [27]:
# Distribution of CabinNum based on CryoSleep
for side in [True, False]:
    print(f"Distribution for CryoSleep = {side}:\n")
    
    # CabinDeck distribution
    cabin_deck_distribution = train_x[train_x['CryoSleep'] == side]['CabinDeck'].value_counts(dropna=False)
    cabin_deck_percentages = (cabin_deck_distribution / cabin_deck_distribution.sum() * 100).round(2)
    
    # Combine counts and percentages into a DataFrame
    cabin_deck_distribution_df = pd.DataFrame({
        'Count': cabin_deck_distribution,
        'Percentage': cabin_deck_percentages
    })
    
    # Calculate percentage relative to total passengers in each CabinDeck
    total_cabin_deck_counts = train_x['CabinDeck'].value_counts(dropna=False)
    cabin_deck_distribution_df['Percentage of Total CabinDeck'] = (
        cabin_deck_distribution / total_cabin_deck_counts * 100
    ).round(2)
    
    print(cabin_deck_distribution_df)

Distribution for CryoSleep = True:

           Count  Percentage  Percentage of Total CabinDeck
CabinDeck                                                  
G           1252       44.44                          54.53
F            503       17.86                          20.09
B            390       13.84                          55.16
C            276        9.80                          41.07
E            163        5.79                          20.40
D             97        3.44                          22.40
NaN           74        2.63                          42.29
A             62        2.20                          26.72
Distribution for CryoSleep = False:

           Count  Percentage  Percentage of Total CabinDeck
CabinDeck                                                  
F           2001       39.97                          79.91
G           1044       20.85                          45.47
E            636       12.70                          79.60
C            396        7.9

Seeing the results we can observe that the same decks exists on both sides, so we are not obtaining any information from that. As I see it, the CabinDeck and CabinNum don't give us any useful information and are categorical variants with lots of different values, so in order to not increase exponentially the complexity of the problem, I will remove them. On the other side, the CabinSide consist in only two different values that we can encode into a single boolean variant. The main problem is what are we going to do with the missing values because I don't have any information about when a passenger is in the Port or the Starboard, I will use an existing Missing Data Imputation method, that I will select afterwards.

Maybe use the CabinDeck info to impute missing CryoSleep values???

In [24]:
# One-hot encoding for HomePlanet
train_x['isCabinPort'] = (train_x['CabinSide'] == 'P').astype(int)

val_x['isCabinPort'] = (val_x['CabinSide'] == 'P').astype(int)

test_x['isCabinPort'] = (test_x['CabinSide'] == 'P').astype(int)

# Verify the new columns
train_x[['CabinSide', 'isCabinPort']].head()

,CabinSide,isCabinPort
4132,P,1
7217,P,1
7216,P,1
7968,S,0
50,S,0


### Destination 
The planet the passenger will be debarking to.

Similar to the HomePlanet, this is a categorical value, so let's again see the class distribution and the number of unique values that exists.

In [28]:
# First I check how many different values we have in this variant (Destination)
print("Destination unique values: ", train_x['Destination'].unique())

# And also if there are any missing values and how many
print("Missing values in Destination: ", train_x['Destination'].isnull().sum(), f"({train_x['Destination'].isnull().sum()/len(train_x)*100:.2f}%)")

Destination unique values:  ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
Missing values in Destination:  166 (2.12%)


First of all, let's solve the issue with the missing values, we can assume that members of the same group have the same destination, so the first approach will be to assign the existing value of one member of the group to other members that have a null value in the Destination variable.

In [33]:
# Check if all members of the same group have the same Destination
group_destination_check = train_x.groupby('Group')['Destination'].nunique()

# Find groups with more than one unique Destination
inconsistent_groups = group_destination_check[group_destination_check > 1]

# Print the result
if inconsistent_groups.empty:
    print("All members of the same group have the same Destination.")
else:
    print("Some groups have inconsistent Destinations:")
    print(inconsistent_groups)

Some groups have inconsistent Destinations:
Group
0008    2
0020    3
0044    2
0067    3
0099    2
       ..
9219    2
9220    2
9227    2
9231    2
9280    2
Name: Destination, Length: 612, dtype: int64


Well, we can't follow this approach now, let's see the class distribution in more detail to see the frequency of each value.

In [35]:
# Class distribution for the Destination column
destination_distribution = train_x['Destination'].value_counts(dropna=False)
destination_percentages = (train_x['Destination'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("Destination distribution (counts):")
print(destination_distribution)
print("\nDestination distribution (percentages):")
print(destination_percentages)

Destination distribution (counts):
Destination
TRAPPIST-1e      5326
55 Cancri e      1616
PSO J318.5-22     715
NaN               166
Name: count, dtype: int64

Destination distribution (percentages):
Destination
TRAPPIST-1e      68.08
55 Cancri e      20.66
PSO J318.5-22     9.14
NaN               2.12
Name: proportion, dtype: float64


Most of the passengers go to TRAPPIST-1e, I could simply give that Destination to all the empty values, however I will better let a Missing Data Imputation method do that job, now let's just do the one-hot encoding to pass this variable to two boolean features that tell if destination isTrappist or isCancri.

In [36]:
# One-hot encoding for Destination
train_x['isTrappist'] = (train_x['Destination'] == 'TRAPPIST-1e').astype(int)
train_x['isCancri'] = (train_x['Destination'] == '55 Cancri e').astype(int)

val_x['isTrappist'] = (val_x['Destination'] == 'TRAPPIST-1e').astype(int)
val_x['isCancri'] = (val_x['Destination'] == '55 Cancri e').astype(int)

test_x['isTrappist'] = (test_x['Destination'] == 'TRAPPIST-1e').astype(int)
test_x['isCancri'] = (test_x['Destination'] == '55 Cancri e').astype(int)

# Verify the new columns
train_x[['Destination', 'isTrappist', 'isCancri']].head()

,Destination,isTrappist,isCancri
4132,TRAPPIST-1e,1,0
7217,TRAPPIST-1e,1,0
7216,PSO J318.5-22,0,0
7968,55 Cancri e,0,1
50,TRAPPIST-1e,1,0


### Age 
The age of the passenger.

This is a numerical variable that doesn't need to apply any encoding, let's see it distribution in detail and if there is any empty value.

In [38]:
# Check the distribution of Age
age_distribution = train_x['Age'].describe()
print("Age distribution:")
print(age_distribution)

# Check for missing values in Age
missing_age_train = train_x['Age'].isnull().sum()

print("\nMissing values in Age:")
print(f"train_x: {missing_age_train} ({missing_age_train / len(train_x) * 100:.2f}%)")

Age distribution:
count    7661.000000
mean       28.829526
std        14.517979
min         0.000000
25%        19.000000
50%        27.000000
75%        38.000000
max        79.000000
Name: Age, dtype: float64

Missing values in Age:
train_x: 162 (2.07%)


Same as the other features, we have a ~2% of empty values, again we are not able to manually impute any of these values, so I do nothing here.

### VIP 
Whether the passenger has paid for special VIP service during the voyage. This is a boolean feature. 

In [39]:
# Print the distribution of VIP
VIP_distribution = train_x['VIP'].value_counts(dropna=False)
VIP_percentages = (train_x['VIP'].value_counts(normalize=True, dropna=False) * 100).round(2)

print("VIP distribution (counts):")
print(VIP_distribution)
print("\nVIP distribution (percentages):")
print(VIP_percentages)

print("Missing values in VIP: ", train_x['VIP'].isnull().sum(), f"({train_x['VIP'].isnull().sum()/len(train_x)*100:.2f}%)")

VIP distribution (counts):
VIP
False    7455
NaN       189
True      179
Name: count, dtype: int64

VIP distribution (percentages):
VIP
False    95.30
NaN       2.42
True      2.29
Name: proportion, dtype: float64
Missing values in VIP:  189 (2.42%)


In [49]:
# Filter passengers where CryoSleep is True
cryo_sleep_true = train_x[train_x['CryoSleep'] == 1]

# Calculate the VIP distribution for passengers in CryoSleep
vip_distribution_cryo_sleep = cryo_sleep_true['VIP'].value_counts(dropna=False)
vip_percentages_cryo_sleep = (cryo_sleep_true['VIP'].value_counts(normalize=True, dropna=False) * 100).round(2)

# Print the distribution
print("VIP distribution when CryoSleep is True:")
print(vip_distribution_cryo_sleep)
print(vip_percentages_cryo_sleep)

VIP distribution when CryoSleep is True:
VIP
False    2725
NaN        73
True       19
Name: count, dtype: int64
VIP
False    96.73
NaN       2.59
True      0.67
Name: proportion, dtype: float64


Taking into account that in a normal distribution only ~2.29% of the samples have VIP == True, that means that from our set of NaN values, we will only have 4 or 5 wrong values (Passenger with VIP == True where we set VIP == False), so I will set all empty VIP values to False.

In [50]:
# Fill NaN values in the VIP column with False
train_x['VIP'] = train_x['VIP'].fillna(False)
val_x['VIP'] = val_x['VIP'].fillna(False)
test_x['VIP'] = test_x['VIP'].fillna(False)

# Verify if there are still missing values in VIP
print("Missing values in VIP after filling with False:")
print(f"train_x: {train_x['VIP'].isnull().sum()}")
print(f"val_x: {val_x['VIP'].isnull().sum()}")
print(f"test_x: {test_x['VIP'].isnull().sum()}")

Missing values in VIP after filling with False:
train_x: 0
val_x: 0
test_x: 0


C:\Users\javie\AppData\Local\Temp\ipykernel_284\1786103325.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_x['VIP'] = train_x['VIP'].fillna(False)
C:\Users\javie\AppData\Local\Temp\ipykernel_284\1786103325.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_x['VIP'] = val_x['VIP'].fillna(False)
C:\Users\javie\AppData\Local\Temp\ipykernel_284\1786103325.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to 

### RoomService, FoodCourt, ShoppingMall, Spa, VRDeck 
Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities. 

These are all numerical features, let's check their distribution and if there are missing values.

In [51]:
# Check the distribution and missing values for RoomService, FoodCourt, ShoppingMall, Spa, and VRDeck
amenities = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

for amenity in amenities:
    print(f"Distribution for {amenity}:")
    print(train_x[amenity].describe())
    print(f"\nMissing values in {amenity}: {train_x[amenity].isnull().sum()} ({train_x[amenity].isnull().mean() * 100:.2f}%)\n")

Distribution for RoomService:
count     7658.000000
mean       226.560851
std        673.028616
min          0.000000
25%          0.000000
50%          0.000000
75%         50.000000
max      14327.000000
Name: RoomService, dtype: float64

Missing values in RoomService: 165 (2.11%)

Distribution for FoodCourt:
count     7654.000000
mean       457.270577
std       1585.986752
min          0.000000
25%          0.000000
50%          0.000000
75%         79.000000
max      27723.000000
Name: FoodCourt, dtype: float64

Missing values in FoodCourt: 169 (2.16%)

Distribution for ShoppingMall:
count     7635.000000
mean       176.503602
std        620.864161
min          0.000000
25%          0.000000
50%          0.000000
75%         27.000000
max      23492.000000
Name: ShoppingMall, dtype: float64

Missing values in ShoppingMall: 188 (2.40%)

Distribution for Spa:
count     7655.000000
mean       313.306989
std       1141.689289
min          0.000000
25%          0.000000
50%          0.0